In [1]:
!python -m pip install semantic-kernel==1.0.4
!python -m pip install azure-core==1.30.1
!python -m pip install azure-search-documents==11.4.0
!python -m pip install python-dotenv
!python -m pip install azure.identity

In [6]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aiSerchEndpoint =os.getenv("AZURE_AI_SEARCH_ENDPOINT")
aiSearchKey = os.getenv("AZURE_AI_SEARCH_API_KEY")


In [8]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureAISearchDataSource,
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
    ExtraBody,
)
from semantic_kernel.connectors.memory.azure_cognitive_search.azure_ai_search_settings import AzureAISearchSettings
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments
from semantic_kernel.kernel import Kernel
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

kernel = Kernel()
azure_ai_search_settings = AzureAISearchSettings(api_key=aiSearchKey, endpoint=aiSerchEndpoint, index_name="default")

# This example index has fields "title", "chunk", and "vector".
# Add fields mapping to the settings.
azure_ai_search_settings["fieldsMapping"] = {
    "titleField": "id",
    "contentFields": ["payload"],
    "vectorFields": ["embedding"],
}

# Add Ada embedding deployment name to the settings and use vector search.
azure_ai_search_settings["embeddingDependency"] = {
    "type": "DeploymentName",
    "deploymentName": "embedding",
}
azure_ai_search_settings["query_type"] = "vector"

# Create the data source settings
az_source = AzureAISearchDataSource(parameters=azure_ai_search_settings.model_dump())
extra = ExtraBody(data_sources=[az_source])
service_id = "chat-gpt"
req_settings = AzureChatPromptExecutionSettings(service_id=service_id, extra_body=extra)

# When using data, use the 2024-02-15-preview API version.
chat_service = AzureChatCompletion(
    service_id="chat-gpt",
)
kernel.add_service(chat_service)

prompt_template_config = PromptTemplateConfig(
    template="{{$chat_history}}{{$user_input}}",
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="chat_history", description="The history of the conversation", is_required=True, default=""),
        InputVariable(name="request", description="The user input", is_required=True),
    ],
    execution_settings=req_settings,
)

chat_history = ChatHistory()

chat_history.add_user_message("Hi there, who are you?")
chat_history.add_assistant_message("I am an AI assistant here to answer your questions.")

chat_function = kernel.add_function(
    plugin_name="ChatBot", function_name="Chat", prompt_template_config=prompt_template_config
)

user_input = "Do you know anything about Orion?"

arguments = KernelArguments(chat_history=chat_history, user_input=user_input, execution_settings=req_settings)

full_message = None
print("Assistant:> ", end="")
async for message in kernel.invoke_stream(chat_function, arguments=arguments):
    print(str(message[0]), end="")
    full_message = message[0] if not full_message else full_message + message[0]
chat_history.add_assistant_message(str(full_message))
print("\n")

TypeError: 'AzureAISearchSettings' object does not support item assignment